In [1]:
import csv
import pandas as pd
import tensorflow as tf
import numpy as np
LABELS_2 = {'Benign':0,
          'SSH-Bruteforce':1,
          'FTP-BruteForce':1,
          'Brute Force -Web':1,
          'Brute Force -XSS':1,
          'DDOS attack-HOIC':1,
          'DoS attacks-Slowloris':1,
          'DDOS attack-LOIC-UDP':1,
          'DoS attacks-Hulk':1,
          'DoS attacks-GoldenEye':1,
          'DoS attacks-SlowHTTPTest':1,
          'DDoS attacks-LOIC-HTTP':1,
          'Bot':1,
          'SQL Injection':1,
          'Infilteration':1,
          }
LABELS_15 = {'Benign':0,
          'SSH-Bruteforce':1,
          'FTP-BruteForce':2,
          'Brute Force -Web':3,
          'Brute Force -XSS':4,
          'DDOS attack-HOIC':5,
          'DoS attacks-Slowloris':6,
          'DDOS attack-LOIC-UDP':7,
          'DoS attacks-Hulk':8,
          'DoS attacks-GoldenEye':9,
          'DoS attacks-SlowHTTPTest':10,
          'DDoS attacks-LOIC-HTTP':11,
          'Bot':12,
          'SQL Injection':13,
          'Infilteration':14,
          }

LABELS_7 = {'Benign':0,#normal
            
            #bruteforce
            'SSH-Bruteforce':1,
            'FTP-BruteForce':1,
            
            #Web
            'Brute Force -Web':2,
            'Brute Force -XSS':2,
            'SQL Injection':2,
            
            #DDOS
            'DDOS attack-HOIC':3,
            'DDOS attack-LOIC-UDP':3,
            'DDoS attacks-LOIC-HTTP':3,
            
            #dos
            'DoS attacks-Slowloris':4,
            'DoS attacks-Hulk':4,
            'DoS attacks-GoldenEye':4,
            'DoS attacks-SlowHTTPTest':4,
            
            
            'Bot':5,
            
            
            'Infilteration':6,
          }

filenames = ['Friday-02-03-2018_TrafficForML_CICFlowMeter.csv', #0
             'Friday-16-02-2018_TrafficForML_CICFlowMeter.csv',
             'Friday-23-02-2018_TrafficForML_CICFlowMeter.csv',
             'Thursday-01-03-2018_TrafficForML_CICFlowMeter.csv',
             'Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv',
             'Thursday-22-02-2018_TrafficForML_CICFlowMeter.csv',
             'Wednesday-14-02-2018_TrafficForML_CICFlowMeter.csv',
             'Wednesday-21-02-2018_TrafficForML_CICFlowMeter.csv',
             'Wednesday-28-02-2018_TrafficForML_CICFlowMeter.csv', #8
             'Thuesday-20-02-2018_TrafficForML_CICFlowMeter.csv'
            ]

def convert_file_labels():
    for f in filenames:
        df=pd.read_csv(f)
        df['Label']=df['Label'].replace([k for k,w in LABELS_7.items()], [w for k,w in LABELS_7.items()])
        df.to_csv( "7_classes/%s"%f, index=False, encoding='utf-8-sig')

In [ ]:
def get_data(class_num=7):
    if class_num==7:
        li = []

        for filename in filenames:
            df = pd.read_csv("7_classes/processed/%s"%filename, index_col=None)
            li.append(df)

        frame = pd.concat(li, axis=0, ignore_index=True)
        frame.pop("Unnamed: 0")
        return frame

def preprocess_data(frame, class_num=7, batch_size=128):
    rnd_seed=1
    train_li, validate_li, test_li = [], [], []
    
    for i in range(class_num):
        print(i)
        count = len(frame[frame['Label']==i])
        if count>2000000:
            count = 2000000
        train_count = int(0.7 * count)
        test_count = int(0.15 * count)
        validate_count = int(0.15 * count)
        train_li.append(frame[frame['Label']==i].sample(n=count,random_state=rnd_seed)[0:train_count])
        validate_li.append(frame[frame['Label']==i].sample(n=count,
                                                           random_state=rnd_seed)[train_count:train_count+validate_count])
        test_li.append(frame[frame['Label']==i].sample(n=count,random_state=rnd_seed)[train_count+validate_count:count])
    
    
    #save these frames as .csv to fast load dataset
    train_frame = pd.concat(train_li, axis=0, ignore_index=True)
    validate_frame = pd.concat(validate_li, axis=0, ignore_index=True)
    test_frame = pd.concat(test_li, axis=0, ignore_index=True)
    
    
    train_labels = train_frame.pop("Label")
    validate_labels = validate_frame.pop("Label")
    test_labels = test_frame.pop("Label")

    train_dataset = tf.data.Dataset.from_tensor_slices((train_frame.values, 
                                                        tf.keras.utils.to_categorical(train_labels, 
                                                                                      class_num))).batch(batch_size)
    test_dataset = tf.data.Dataset.from_tensor_slices((test_frame.values, 
                                                        tf.keras.utils.to_categorical(test_labels, 
                                                                                      class_num))).batch(batch_size)
    val_dataset = tf.data.Dataset.from_tensor_slices((validate_frame.values, 
                                                        tf.keras.utils.to_categorical(validate_labels, 
                                                                                      class_num))).batch(batch_size)
    return (train_dataset, val_dataset, test_dataset)

def train_model(model, rnd_seed=1):
    log_dir="./logs/fit/" +str(rnd_seed)+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1) 
    
    data = get_data()
    train_dataset ,val_dataset, test_dataset = preprocess_data(data)
    
    train_history = model.fit(train_dataset,
              epochs=epochs,
              validation_data=val_dataset,
              callbacks=[tensorboard_callback]
             )
    
    model.evaluate(test_dataset, callbacks=[tensorboard_callback])
    model.save(savename)
    log.info('trained on file %s: model saved with name %s'%(filename,savename))

In [52]:
convert_file_labels()

c:\intelpython3\lib\site-packages\IPython\core\interactiveshell.py:3242: DtypeWarning: Columns (16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
c:\intelpython3\lib\site-packages\IPython\core\interactiveshell.py:3242: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
c:\intelpython3\lib\site-packages\IPython\core\interactiveshell.py:3242: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
c:\intelpython3\lib\site-packages\IPython\core\interactiveshell.py:3242: DtypeWarning: C

In [5]:
for f in filenames:
    df=pd.read_csv("7_classes/processed/%s"%f)
#     df['Flow Byts/s']=df['Flow Byts/s'].replace(['Infinity'], 9999999999.99).astype('float64')
#     df['Flow Byts/s']=df['Flow Byts/s'].replace([np.nan], -1.0).astype('float64')
#     df['Flow Pkts/s']=df['Flow Pkts/s'].replace(['Infinity'], 9999999999.99).astype('float64')
#     df['Flow Pkts/s']=df['Flow Pkts/s'].replace([np.nan], -1.0).astype('float64')
    #     df.pop('Flow Byts/s')
    #     df.pop('Flow Pkts/s')
#     df.pop('Timestamp')
#     if f=='Thuesday-20-02-2018_TrafficForML_CICFlowMeter.csv':
#         df.pop("Flow ID")
#         df.pop("Src Port")
#         df.pop("Src IP")
#         df.pop("Dst IP")
    df['Label']=df['Label'].replace([1,2,3,4,5,6],1)
    df.to_csv("2_classes/processed/%s"%f)
    print(f)

Friday-02-03-2018_TrafficForML_CICFlowMeter.csv
Friday-16-02-2018_TrafficForML_CICFlowMeter.csv
Friday-23-02-2018_TrafficForML_CICFlowMeter.csv
Thursday-01-03-2018_TrafficForML_CICFlowMeter.csv
Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv
Thursday-22-02-2018_TrafficForML_CICFlowMeter.csv
Wednesday-14-02-2018_TrafficForML_CICFlowMeter.csv
Wednesday-21-02-2018_TrafficForML_CICFlowMeter.csv
Wednesday-28-02-2018_TrafficForML_CICFlowMeter.csv
Thuesday-20-02-2018_TrafficForML_CICFlowMeter.csv


In [8]:
train_frame = pd.concat(train_li, axis=0, ignore_index=True)
validate_frame = pd.concat(validate_li, axis=0, ignore_index=True)
test_frame = pd.concat(test_li, axis=0, ignore_index=True)

In [9]:
for i in range(2):
    print(len(train_frame[train_frame["Label"]==i]))
    print(len(validate_frame[validate_frame["Label"]==i]))
    print(len(test_frame[test_frame["Label"]==i]))
    print("-------------")

9439295
2022706
2022707
-------------
1923764
412235
412236
-------------


In [19]:
train_frame.to_csv('train2class.csv')
validate_frame.to_csv('val2class.csv')
test_frame.to_csv('test2class.csv')

In [ ]:
train_labels = train_frame.pop("Label")
validate_labels = validate_frame.pop("Label")
test_labels = test_frame.pop("Label")

train_dataset = tf.data.Dataset.from_tensor_slices((train_frame.values, 
                                                    tf.keras.utils.to_categorical(train_labels, num_classes)))
test_dataset = tf.data.Dataset.from_tensor_slices((test_frame.values, 
                                                    tf.keras.utils.to_categorical(test_labels, num_classes)))
val_dataset = tf.data.Dataset.from_tensor_slices((validate_frame.values, 
                                                    tf.keras.utils.to_categorical(validate_labels, num_classes)))

In [84]:
frame[frame['Label']==2].sample(n=10,random_state=3)

,Unnamed: 0,Dst Port,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
2098490,1341,80,6,57329869,203,104,56330.0,190010.0,680.0,0.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
2097609,460,80,6,5008541,4,4,646.0,364.0,646.0,0.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
4526689,1290,80,6,175,3,0,0.0,0.0,0.0,0.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
2098390,1241,80,6,57370027,203,104,56330.0,190022.0,680.0,0.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
4525993,594,80,6,5008347,4,4,646.0,364.0,646.0,0.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
4525876,477,80,6,56637264,153,104,54995.0,72482.0,646.0,0.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
2097842,693,80,6,5005914,5,3,646.0,364.0,646.0,0.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
2098719,1570,80,6,5002444,3,1,0.0,0.0,0.0,0.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
4526004,605,80,6,57935079,153,104,54995.0,72478.0,646.0,0.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
2098433,1284,80,6,116,3,0,0.0,0.0,0.0,0.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2


In [1]:
from imblearn.over_sampling import SMOTE
import pandas as pd
import tensorflow as tf
from collections import Counter

Using TensorFlow backend.


In [11]:
test_frame = pd.read_csv("test.csv", index_col=None)
print("test_frame loaded")

test_labels = test_frame.pop("Label")

test_x = test_frame.values#.reshape([-1,78,1]) 
test_y = test_labels#tf.keras.utils.to_categorical(test_labels, 7)

print("dataset loaded")

test_frame loaded
dataset loaded


In [16]:
oversample = SMOTE(sampling_strategy = {2:30000})
x, y = oversample.fit_resample(test_x, test_y)
counter = Counter(y)
print(counter)

Counter({0: 300000, 3: 189591, 4: 98145, 1: 57143, 5: 42930, 2: 30000, 6: 24291})


In [3]:
train_frame = pd.read_csv("train.csv", index_col=None)
train_frame = train_frame.sample(n=len(train_frame))

validate_frame = pd.read_csv("val.csv", index_col=None)

test_frame = pd.read_csv("test.csv", index_col=None)


train_labels = train_frame.pop("Label")
validate_labels = validate_frame.pop("Label")
test_labels = test_frame.pop("Label")

tr = Counter(train_labels)
va = Counter(validate_labels)
te = Counter(test_labels)
print(tr)
print(va)
print(te)

Counter({0: 1400000, 3: 884753, 4: 458010, 1: 266664, 5: 200333, 6: 113353, 2: 649})
Counter({0: 300000, 3: 189589, 4: 98145, 1: 57142, 5: 42928, 6: 24290, 2: 139})
Counter({0: 300000, 3: 189591, 4: 98145, 1: 57143, 5: 42930, 6: 24291, 2: 140})


In [13]:
counter = Counter(test_y)
print(counter)

Counter({0: 300000, 3: 189591, 4: 98145, 1: 57143, 5: 42930, 6: 24291, 2: 140})


In [6]:
def load_tmp_dataset(class_num=2):
    train_frame = pd.read_csv("train.csv", index_col=None)
    train_frame = train_frame.sample(n=len(train_frame))
#     log.info("train_frame loaded")
    validate_frame = pd.read_csv("val.csv", index_col=None)
#     log.info("validate_frame loaded")
    test_frame = pd.read_csv("test.csv", index_col=None)
#     log.info("test_frame loaded")
    
    train_frame['Label'] = train_frame['Label'].replace([1,2,3,4,5,6], 1)
    validate_frame['Label'] = validate_frame['Label'].replace([1,2,3,4,5,6], 1)
    test_frame['Label'] = test_frame['Label'].replace([1,2,3,4,5,6], 1)
    
    train_labels = train_frame.pop("Label")
    validate_labels = validate_frame.pop("Label")
    test_labels = test_frame.pop("Label")
    
    train_frame.pop("Dst Port")
    validate_frame.pop("Dst Port")
    test_frame.pop("Dst Port")

    train_x = train_frame.values
    train_y = tf.keras.utils.to_categorical(train_labels, class_num)
    
    
    #oversample = SMOTE(sampling_strategy = sampling_parameters)
    #train_x, train_y = oversample.fit_resample(x, y)
    #train_x = train_x.reshape([-1,77,1])
#     train_y = tf.keras.utils.to_categorical(train_y, class_num)
    
    
    test_x = test_frame.values#.reshape([-1,77,1]) 
    test_y = tf.keras.utils.to_categorical(test_labels, class_num)
    
    val_x = validate_frame.values#.reshape([-1,77,1])
    val_y = tf.keras.utils.to_categorical(validate_labels, class_num)
#     log.info("dataset loaded")
    return (train_x, train_y, val_x, val_y, test_x, test_y)